# Challenge MeLi

In [1]:
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import numpy as np
import time
import math
from random import sample
import gc


#del variable
#gc.collect()

In [7]:
#df_train = pd.read_csv('./data/03_primary/train_dataset.csv')
#df_test = pd.read_csv('./data/03_primary/test_dataset.csv')
dtype ={"item_id": "int", "title": "str", "domain_id": "str", "price": "float", "category_id": "str", "condition": "str", "site": "str"}
df_item = pd.read_csv('./data/02_intermediate/item_data.csv', dtype=dtype)
#df_item.set_index('item_id')
df_train = pd.read_csv('./data/02_intermediate/train_reduce_1.csv')

In [10]:
from pandas_utils import extended_describe
extended_describe(df_item, 2, ['O'])

,title,domain_id,category_id,condition,site
count,2102277,2102277,2102277,2102277,2102277
unique,1925296,7894,11493,3,2
top,Xiaomi Redmi Note 7 (48 Mpx) Dual Sim 64 Gb Sp...,MLB-CARS_AND_VANS,MLB1744,new,MLB
freq,143,41420,41204,1866333,1723826
missing,0,0,0,0,0


In [11]:
extended_describe(df_item, 2)

,item_id,price
count,2102277.00,2.102277e+06
mean,1051138.00,1.348849e+05
std,606875.24,2.299328e+07
min,0.00,-1.000000e+00
25%,525569.00,6.990000e+01
50%,1051138.00,1.749000e+02
75%,1576707.00,5.729400e+02
max,2102276.00,1.000000e+10
missing,0.00,0.000000e+00


In [12]:
list_of_domains = df_item.domain_id.unique().tolist()
list_of_domains.sort()

In [13]:
items_view_by_session = {}
for name, group in tqdm_notebook(df_train.groupby('session_id')):
    list_of_items = group[group.event_type == "view"].event_info.unique().tolist()
    items_view_by_session[name] = list_of_items

In [14]:
temp = df_train[['session_id', 'event_info', 'event_type']]
temp = temp[df_train.event_type == 'view']
temp.drop(columns='event_type', inplace=True)
temp.event_info = pd.to_numeric(temp.event_info)

In [15]:
temp_merge = temp.merge(df_item[['item_id', 'domain_id']], left_on='event_info', right_on='item_id')[['session_id', 'domain_id']]


In [16]:
items_view_by_session = {}
for name, group in tqdm_notebook(temp_merge.groupby('session_id')):
    items_view_by_session[name] = group.domain_id.unique().tolist()

In [ ]:
set_of_values = set()
for elem in items_view_by_session.values():
    for el in elem:
        set_of_values.add(el)

In [ ]:
len(set_of_values)

In [ ]:
list(items_view_by_session.values())[0]

### Base line tomando algun elemento de la lista que vio

In [66]:
def dcg(predicted, effective_item):
    dcg = 0
    #map(lambda p, e: relevance(p, e, items), predicted, effective)
    for pos, (item, domain) in enumerate(predicted.items()):
        dcg += relevance((item, domain), effective_item) / (math.log10(2 + pos))
    return dcg


def relevance(predicted_item, effective_item):
    if predicted_item[0] == effective_item[0]:
        return 12
    elif (predicted_item[1] == effective_item[1]):
        return 1
    else:
        return 0


def idcg(number_of_items=10):
    # 12/math.log10(1 + 1) + 1/math.log10(1 + 2) + 1/math.log10(1 + 3) .... 
    first = 12 / math.log10(1 + 1)
    all_except_first = sum(map(lambda position: 1 / math.log10(1 + position), range(2, number_of_items + 1)))
    return first + all_except_first


def ndcg(predicted, effective_item, items):
    predicted_dict = {}
    for one_item in predicted:
        predicted_dict[one_item] = items[items.item_id == one_item].domain_id.iloc[0]
    effective_item = (
        effective_item, items[items.item_id == effective_item].domain_id.iloc[0]
    )
    return (dcg(predicted_dict, effective_item) / idcg())


In [17]:
def split_train_test_ids(list_of_ids, percentage=70):
    total = len(list_of_ids) * (percentage / 100)
    session_id_train = sample(list_of_ids, int(total))
    session_id_test = list(filter(lambda id: id not in session_id_train, list_of_ids))
    return session_id_train, session_id_test

In [18]:
train_id, test_id = split_train_test_ids(df_train.session_id.unique().tolist(), 70)

In [19]:
train = df_train[df_train.session_id.isin(train_id)]

In [74]:
# genero un diccionario por session con otro diccionario que tiene el id del item visto y la cantidad de veces que lo vio. Ordenado desde el último visto como primero del diccionario
# {session_id: {item_id: how_many_times_saw}, ....}
recommended_by_session = {}
for id, group in tqdm_notebook(train.groupby('session_id')):
    session_records = train[train.session_id==id]
    last_items_view = group[group.event_type == 'view'].event_info.tolist()
    last_items_view.reverse()
    last_items_view = list(map(lambda item: int(item), last_items_view))
    last_items_view_quantity = {}
    for an_item in last_items_view:
        if an_item in last_items_view_quantity:
            last_items_view_quantity[an_item] += 1
        else:
            last_items_view_quantity[an_item] = 1
    
    recommended_by_session[id] = last_items_view_quantity


In [75]:
# como los primeros predictos tomo la lista de items vistos sin repetir
items = recommended_by_session[4]
predicted = list(items.keys())[:10]
predicted

[1122777, 1994650, 2082574, 1214201, 1940290, 823109, 1352620, 1838988]

In [23]:
# ordeno por cantidad de vistas
sorted_items = {k: v for k, v in sorted(items.items(), key=lambda item: item[1], reverse=True)}
most_viewed = list(sorted_items.keys())[0]
most_viewed

1122777

In [24]:
# categoria mas vista
most_viewed_by_domain = df_item[df_item.item_id == most_viewed].domain_id.iloc[0]
most_viewed_by_domain

'MLM-CELLPHONE_COVERS'

In [25]:
# elijo items al azar que pertenzcan a la categoria mas vista
predicted += df_item[df_item.domain_id == most_viewed_by_domain][:10 - len(predicted)].index.tolist()
predicted

[1122777, 1994650, 2082574, 1214201, 1940290, 823109, 1352620, 1838988, 6, 13]

In [73]:
ndcg(predicted, 1122777, df_item)

0.793469054614242

In [104]:
df_item.loc[randint(0, len(df_item) -1 )].item_id

1725842

In [78]:
df_item.head()

,item_id,title,domain_id,price,category_id,condition,site
0,111260,Casa Sola En Venta Con Gran Patio Solo Pago De...,MLM-INDIVIDUAL_HOUSES_FOR_SALE,1150000.00,MLM170527,new,MLM
1,871377,Resident Evil Origins Collection Nintendo Swit...,MLM-VIDEO_GAMES,1392.83,MLM151595,new,MLM
2,490232,Falda De Imitación Piel Negra,MLM-SKIRTS,350.00,MLM7697,new,MLM
3,1150706,Powercolor Red Devil Radeon Rx 580 8gb Gddr5,MLM-GRAPHICS_CARDS,3200.00,MLM9761,used,MLM
4,934912,Laptop Hp Nx6320 Core Duo Con Puerto Db9 Windo...,MLM-NOTEBOOKS,1599.00,MLM1652,used,MLM


In [126]:
with open('./data/07_model_output/submission_baseline_1.csv') as file:
    for pos, line in enumerate(file.read().splitlines()):
        split_line = line.split(',')
        if len(split_line) != 10:
            print(pos, len(split_line))

In [118]:
for i in range(0):
    print(i)

In [111]:
b.split(',')

['1615991',
 '1786148',
 '1661244',
 '515041',
 '1516428',
 '1336169',
 '113161',
 '1576116',
 '357137',
 '1131224\n']

In [122]:
b=[1]
b.append(2)
b

[1, 2]